In [ ]:
import pandas as pd
import time
from google import genai
from data import read_data
from datetime import datetime
import math


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

In [ ]:
# data = pd.read_csv("smile-college-dataset.csv")
# data.drop(data.columns[0], axis=1, inplace=True)
data =read_data()
data 

In [ ]:
df = data.copy()
count = 0
batch_size = 100
start_index = 0
max_rows = len(df)
num_batches = math.ceil((max_rows-start_index)/batch_size)
client = genai.Client()


for i in range(num_batches):
    end_index = min(start_index+batch_size, max_rows)
    print(f"Processing rows {start_index} to {end_index - 1}")
    query_json = df.iloc[start_index:start_index+batch_size]["comment"].to_dict()
    prompt = f"""
            You are an experienced sentiment analyst studying students’ feedback on mental health services.
            You will receive a python dictionary containing index as key and response as value.
            Categorize the each of the following response into one of these four labels: Satisfied, Dissatisfied, Mixed, Neutral.

            The specific criteria for each category are as follows. 
            (1) “Satisfied”: at least 75% of the language expressed satisfaction, with minimal suggestions for improvement. 
            (2) “Dissatisfied”: at least 75% of the language indicated discontent or suggestions for enhancement, with little mention of satisfaction. 
            (3) “Mixed”: expressions of satisfaction and dissatisfaction/suggestions were approximately evenly split, with each constituting about 50%. 
            (4) “Neutral”: no clear emphasis on satisfaction, dissatisfaction, or suggestions for improvement

            Question: “What mental health or wellness services and supports provided by your college are working well? What aspects need more attention?”
            Response: {query_json}
            
            Return your output **strictly** as a dictionary in the same order as the input.

            Output: 
            """
        
    response = client.models.generate_content(
            model="gemini-2.5-flash", contents=prompt
        )
    print(response.text)
    # labels_dict = json.loads(response)
    # df["Predicted_labels"] = df.index.map(labels_dict)
    start_index = end_index
    break
        

In [ ]:
prompt_out = {
    300: "Mixed",
    301: "Satisfied",
    302: "Mixed",
    303: "Mixed",
    304: "Dissatisfied",
    305: "Neutral",
    306: "Mixed",
    307: "Neutral",
    308: "Mixed",
    309: "Mixed",
    310: "Dissatisfied",
    311: "Mixed",
    312: "Neutral",
    313: "Mixed",
    314: "Neutral",
    315: "Mixed",
    316: "Dissatisfied",
    317: "Dissatisfied",
    318: "Mixed",
    319: "Dissatisfied",
    320: "Satisfied",
    321: "Mixed",
    322: "Mixed",
    323: "Mixed",
    324: "Dissatisfied",
    325: "Dissatisfied",
    326: "Mixed",
    327: "Dissatisfied",
    328: "Dissatisfied",
    329: "Mixed"
}


In [ ]:
output = pd.read_csv("result copy.csv")
output = output.iloc[300:330][["Predicted_labels"]]
# output = output.reset_index(drop=True)
output

In [ ]:
prompt_out_df = pd.DataFrame.from_dict(prompt_out, orient='index', columns=["Predicted values"])
prompt_out_df

In [ ]:
# Normalize and align
o = output["Predicted_labels"].astype(str).str.strip().str.lower()
p = prompt_out_df["Predicted values"].astype(str).str.strip().str.lower()

# Find differing rows
diff = output[o != p]
print(diff)
